# CP101: Tutorial 17 Q2


Formaldehyde (CH$_2$O) is produced by the catalytic oxidation of
methanol (CH$_3$OH) using an excess of air.  Water is also produced in
the reaction.  Under certain conditions, a secondary reaction occurs
in which some formaldehyde is further oxidised to formic acid
(HCOOH). In a test run on a pilot plant, the product gases are found
to have the following composition by volume:


| CH$_3$ OH | CH$_2$ O | HCOOH | H$_2$ O | O$_2$ | N$_2$ |
| ---: | ---: | ---: | ---: | ---: | ---: |
|      9.9% |     9.9% |  4.9% |   14.8% |  4.9% | 55.6% |


Calculate: 
  (a) The molar ratio of feed air to feed methanol.
  (b) The percentage of formaldehyde lost in the secondary reaction.


## Atom balances

To begin solving the problem, we draw a process flow diagram for the furnace, labeling each of the streams and giving their overall flow rates and compositions.

![control volume](./CP101T17Q2.png)

We take $A$ to be the molar flow rate of methanol, $B$ to be the flow rate of air supplied, and $C$ to be molar flow rate of the product gas.  In the calculations, we take a basis where the total product gas flow rate is $100\,{\rm mol\,h^{-1}}$.  We take the convention that a positive flow implies that material is entering the system, while a negative flow implies that material is exiting the system; therefore, we have $C=-100\,{\rm mol\,h^{-1}}$.  


In principle we can write the equations for the four atom balances, however, we note that two of the balances are redundant, since we have four equations and only two unknowns $A$ and $B$.

Nitrogen balance:
\begin{align*}
0.79 B + 2(0.556) C &= 0
\end{align*}

Carbon balance:
\begin{align*}
A + 0.099C + 0.099C + 0.049C &= 0
\end{align*}

Oxygen balance:
\begin{align*}
A + 2(0.21B) + 0.099C + 0.099C + 2(0.049)C + 0.148C + 2(0.049)C &= 0
\end{align*}

Hydrogen balance:
\begin{align*}
4A - 4(0.099C) + 2(0.099C) + 2(0.049C) + 2(0.148)C &= 0
\end{align*}

In [7]:
data = {}
data['CH3OH'] = {'C':1, 'H':4, 'O':1}
data['CH2O'] = {'C':1, 'H':2, 'O':1}
data['HCOOH'] = {'C':1, 'H':2, 'O':2}                
data['CH4'] = {'C':1, 'H':4}
data['N2'] = {'N':2}
data['C2H6'] = {'C':2, 'H':6}
data['CO2'] = {'C':1, 'O':2}
data['CO'] = {'C':1, 'O':1}
data['O2'] = {'O':2}
data['H2O'] = {'H':2, 'O':1}


streamA = {}
streamA['dotN'] = 0.0
streamA['species'] = {'CH3OH':1.0}

streamB = {}
streamB['dotN'] = 0.0
streamB['species'] = {'N2':0.79, 'O2':0.21}


streamC = {}
streamC['dotN'] = -100.0
streamC['species'] = {'CH3OH':0.099, 
                      'CH2O':0.099, 'H2O':0.148, 
                      'O2':0.07,'N2':0.85}

stream_list = [streamA, streamB, streamC]


A = 0.0
B = 0.0

streamA['dotN'] = A
streamB['dotN'] = B

res = []
for atom in ['N', 'C']:
    accumulation = 0.0
    print(atom + ' balance:')
    for stream in stream_list:
        flowrate = stream['dotN']
        for molecule, x in stream['species'].items():
            if (atom in data[molecule]):
                natoms = data[molecule][atom]
                print(molecule, atom, natoms)
                accumulation += natoms*x*flowrate
        print(accumulation)
        res.append(accumulation)



N balance:
0.0
N2 N 2
0.0
N2 N 2
-170.0
C balance:
CH3OH C 1
0.0
0.0
CH3OH C 1
CH2O C 1
-19.8


## Python code

The element balances given above provide four equations which can be solved to determine the four unknowns: $A$, $B$, $C$, and $F_{\rm H_2O}$.  However, performing the algebra by hand is rather tedious.  Instead, we can use Python to solve these equations.


In [8]:
from scipy.optimize import fsolve


def acc(x):

    A, B = x
    
    streamA['dotN'] = A
    streamB['dotN'] = B

    res = []
    for atom in ['N', 'C']:
        accumulation = 0.0
        for stream in stream_list:
            flowrate = stream['dotN']
            for molecule, x in stream['species'].items():
                if (atom in data[molecule]):
                    natoms = data[molecule][atom]
                    accumulation += natoms*x*flowrate
        res.append(accumulation)
    
    return res



# make initial guess
A = 100.0
B = 0.0
x0 = [A, B]

# solve the equations
solution = fsolve(acc, x0)

print(solution)


[ 19.8        107.59493671]
